In [1]:
import param as pm
import panel as pn

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
class Wallet(pm.Parameterized):
    total_fund = pm.Number(0, bounds=(0, None), doc="wallet amount in USD")
    total_ctr = pm.Number(0, bounds=(0, None), doc="Amount of CTR tokens in wallet")
    
    def __init__(self, **params):
        super().__init__(**params)
    
    def add_funds(self, amount):
        self.total_fund += amount
        
    def add_ctr(self, ctr):
        self.total_ctr += ctr
        
    def return_funds(self):
        return self.total_fund
    
    def return_ctr(self):
        return self.total_ctr

In [7]:
class OldGrowthForestUnit(pm.Parameterized):
    """OGFU"""
    tree_century_count = pm.Number(32000, bounds=(0,1e6), doc="TCC")
    cost_for_protection = pm.Number(320000, bounds=(0, 1e6), doc="Cost for Protection")
    token_multiplier = pm.Number(1, bounds=(0, 10), doc="TM")
    staked_funds = pm.Number(0, bounds=(0, None), doc="The amount staked in a respective OGFU in CTR")
    staked_amount = pm.Number(0, bounds=(0, None), doc="The amount released after an OGFU is fully staked")
    approved = pm.Boolean(False, doc="Boolean that determines whether OGFU has been approved by Centree fund")
    full_stake = pm.Boolean(False, doc="Boolean that determines when the OGFU is fully staked")
    
    def __init__(self, **params):
        super().__init__(**params)
    
    def check_full_stake(self):
        # Assume OGFU is fully staked at 50000 CTR
        if (self.staked_funds >= 50000):
            self.full_stake = True
        else: 
            self.full_stake = False

    def get_protection_cost_per_century(self):
        """PCC"""
        return self.cost_for_protection / self.tree_century_count
    
    def get_ctr_tokens_issued(self):
        """CTR Tokens Issued"""
        return self.tree_century_count * self.token_multiplier
    
    def stake(self, wallet: Wallet, ctr, total_ctr):
        """Community members vote to protect a particular OGFU by “staking” their CTR against particular
        Tree Centuries in that OGFU. OGFUs can only be staked against when their PCC is lower than 
        FundEquity/TotalCTRIssued. Minimum stake is 100CTR"""
        pcc = self.get_protection_cost_per_century()
        if (ctr > 100) and (pcc < total_ctr):
            # Remove the funded amount from user's wallet & add it to OGFU staked amount
            wallet.add_ctr(-ctr)
            self.staked_funds += ctr
            self.check_full_stake()
            
        return wallet
        
    def generate_revenue(self):
        """A funded OGFU will periodically generate revenue for later distribution"""
        pass

In [8]:
class Centree(pm.Parameterized):
    """Centree"""
    approved_protection_cost_per_century = pm.Number(500, bounds=(0, 10000), doc="APCC")
    centree_general_fund = pm.Number(0, bounds=(0, None), doc="CGF")
    total_ctr = pm.Number(0, bounds=(0, None), doc="Total CTR generated by Fund")
    
    def __init__(self, **params):
        super().__init__(**params)
    
    def get_total_ctr_issued(self):
        """Returns total CTR issued in Centree General Fund"""
        return self.total_ctr
    
    def generate(self, ctr_generated):
        """Creates CTR to add to the existing pool of CTR"""
        self.total_ctr += ctr_generated
    
    def convert_funds_to_ctr(self, funds):
        """Functions that returns CTR equivalent to funds provided"""
        # Assume 1$ => 10 CTR
        ctr = funds * 10
        # Remove CTR from total CTR generated by centree general fund
        if (self.total_ctr < ctr):
            return 0
        self.total_ctr -= ctr
    
        return ctr
    
    def approve(self, ogfu):
        """When an OGFU is approved by Centree for protection, its TCC & PCC are assessed 
        & CC*TM new CTR tokens are issued"""
        # Approve OGFU & generate CTR based on OFGU's TCC
        ogfu.approved = True
        ctr_generated = ogfu.get_ctr_tokens_issued()
        self.generate(ctr_generated)
        
        return ogfu
    
    def buy(self, wallet, funds):
        """A wallet can exchange funds for CTR"""
        # Take funds from the wallet's funds & add CTR tokens in return
        self.centree_general_fund += funds
        wallet.add_funds(-funds)
        ctr_generated = self.convert_funds_to_ctr(funds)
        wallet.add_ctr(ctr_generated)
    
        return wallet

In [12]:
wallet = Wallet()
wallet.add_funds(10000)

In [13]:
wallet.return_funds()

10000

In [14]:
centree_fund = Centree()
ogfu = OldGrowthForestUnit()

In [15]:
ogfu.approved

False

In [16]:
ogfu = centree_fund.approve(ogfu)

In [17]:
ogfu.approved

True

In [18]:
centree_fund.get_total_ctr_issued()

32000

In [19]:
centree_fund.generate(8000)

In [20]:
centree_fund.get_total_ctr_issued()

40000

In [15]:
wallet.add_funds(3000)
wallet.return_funds()

6000

In [21]:
wallet = centree_fund.buy(wallet, 4000)

In [22]:
wallet.return_funds()

6000

In [20]:
centree_fund.centree_general_fund

4000

In [21]:
centree_fund.generate(20000)

In [25]:
total_ctr_fund = centree_fund.get_total_ctr_issued()
wallet = ogfu.stake(wallet, 1000, total_ctr_fund)

In [29]:
wallet.return_ctr()

39000

In [28]:
ogfu.staked_funds

1000